# Quickstart: 3-Line Audit with `from_model()`

This notebook demonstrates the programmatic API for GlassAlpha - auditing a model in 3 lines without YAML configs.

**What you'll learn:**
- How to audit a model programmatically (no YAML files)
- Auto-detection of model types and features
- Inline HTML display in Jupyter
- Full determinism and reproducibility


## Install GlassAlpha

```bash
pip install glassalpha
```


## 1. Train a Simple Model


In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

# Create sample dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, random_state=42)

# Convert to DataFrame
feature_names = [f"feature_{i}" for i in range(10)]
X_df = pd.DataFrame(X, columns=feature_names)

# Add protected attributes
rng = np.random.RandomState(42)
X_df["gender"] = rng.choice(["M", "F"], size=1000)
X_df["age_group"] = rng.choice(["18-30", "31-50", "51+"], size=1000)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.3, random_state=42)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")


In [ ]:
# Extract protected attributes (don't use them for training)
gender_train = X_train.pop("gender")
gender_test = X_test.pop("gender")
age_train = X_train.pop("age_group")
age_test = X_test.pop("age_group")

# Train model (on features only)
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

print(f"Model trained! Accuracy: {model.score(X_test, y_test):.3f}")


## 2. Audit in 3 Lines

No YAML files needed - just pass your model and data!


In [ ]:
import glassalpha as ga

# Encode protected attributes
from sklearn.preprocessing import LabelEncoder

le_gender = LabelEncoder()
le_age = LabelEncoder()

gender_encoded = le_gender.fit_transform(gender_test)
age_encoded = le_age.fit_transform(age_test)

result = ga.audit.from_model(
    model=model,
    X=X_test,
    y=y_test,
    protected_attributes={"gender": gender_encoded, "age_group": age_encoded},
    random_seed=42
)

print(f"Audit complete! Result ID: {result.id[:16]}...")


In [ ]:
result  # Auto-displays audit summary!


## 4. Access Results

The `AuditResult` contains all computed metrics:
```python
print("Performance:")
print(f"  Accuracy: {result.performance['accuracy']:.3f}")
print(f"  Precision: {result.performance['precision']:.3f}")
print(f"  Recall: {result.performance['recall']:.3f}")
print(f"  F1: {result.performance['f1']:.3f}")

print("\nFairness:")
for key, value in result.fairness.items():
    print(f"  {key}: {value:.3f}")

print("\nCalibration:")
for key, value in result.calibration.items():
    print(f"  {key}: {value:.3f}")
```


ation.

new phase 2 OSS roadmap